In [23]:
# Configure
PROJECT_ID = "qwiklabs-gcp-03-e39ec6df7e97"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.5-flash"
DATASET = "aurora_bay_data"

In [34]:
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

# Initialize clients
bq_client = bigquery.Client(project=PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(MODEL_NAME)

In [38]:
def search_faqs(user_query):
    """
    Embeds the user query and searches BigQuery for the most similar FAQ.
    """
    sql = f"""
        SELECT base.question, base.answer, distance
        FROM VECTOR_SEARCH(
            TABLE `aurora_bay_data.faqs_with_embeddings`,
            'q_embedding',  -- ERROR 1: Changed 'embedding' to 'q_embedding' to match your table
            (
                -- Generate embedding for the user's query on the fly
                SELECT ml_generate_embedding_result, content  -- ERROR 2: Changed 'text_embedding' to 'ml_generate_embedding_result'
                FROM ML.GENERATE_EMBEDDING(
                    MODEL `aurora_bay_data.embedding_model`,
                    (SELECT @query AS content),
                    STRUCT(TRUE as flatten_json_output)
                )
            ), top_k => 3)
    """

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("query", "STRING", user_query)
        ]
    )

    results = bq_client.query(sql, job_config=job_config)

    # Format context for the LLM
    context_str = ""
    for row in results:
        context_str += f"FAQ Question: {row.question}\nFAQ Answer: {row.answer}\n\n"

    return context_str

In [39]:
def chat_with_aurora(user_query):
    # 1. Retrieve relevant context from BigQuery
    print(f"Searching Aurora Bay knowledge base for: '{user_query}'...")
    context = search_faqs(user_query)

    if not context:
        return "I couldn't find any specific information about that in the Aurora Bay FAQs."

    # 2. Generate Answer using Gemini
    prompt = f"""
    You are a helpful assistant for the town of Aurora Bay, Alaska.
    Use the following FAQ context to answer the user's question.
    If the answer isn't in the context, say you don't know.

    Context:
    {context}

    User Question: {user_query}
    """

    response = model.generate_content(prompt)
    return response.text

In [40]:
chat_with_aurora("When was the aurora bay founded?")

Searching Aurora Bay knowledge base for: 'When was the aurora bay founded?'...


'Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.'

In [41]:
chat_with_aurora("What is special about this bay?")

Searching Aurora Bay knowledge base for: 'What is special about this bay?'...


'Aurora Bay is special for its unique cultural events, including the Aurora Lights Winter Festival and the Seafarers’ Gathering in August, which celebrates local fishing traditions with boat parades and seafood cook-offs. It is also a popular spot for northern lights viewing in the winter.'

In [42]:
chat_with_aurora("What kind of local fishing traditions is there to do?")

Searching Aurora Bay knowledge base for: 'What kind of local fishing traditions is there to do?'...


'Aurora Bay celebrates local fishing traditions with boat parades and seafood cook-offs during the Seafarers’ Gathering in August.'

In [43]:
chat_with_aurora("What is the best time of the year to visit?")

Searching Aurora Bay knowledge base for: 'What is the best time of the year to visit?'...


"I don't know."

In [44]:
chat_with_aurora("Is it easy to get to this lake? If so, what are the means of transportation?")

Searching Aurora Bay knowledge base for: 'Is it easy to get to this lake? If so, what are the means of transportation?'...


"I don't know. The provided FAQ context does not contain information about getting to a lake or specific transportation methods for lakes."

In [45]:
chat_with_aurora("What about emergencies?")

Searching Aurora Bay knowledge base for: 'What about emergencies?'...


"For emergencies, local residents can sign up for text or email alerts through the town's official website. Important updates are also broadcast via KABY-FM and posted on local social media pages.\n\nFor severe weather emergencies, emergency shelters are typically set up at Aurora Bay High School and the Community Center. The town website and local radio provide real-time updates on shelters."